In [ ]:
import pandas as pd
import optuna
import matplotlib.pyplot as plt

from calibrate import read_data, calc_multi_error
from plot import to_df, get_pareto_front, get_contour_plot, get_parallel_coordinate_plot

%matplotlib inline

In [ ]:
storage = "sqlite:///calibration.db"
print("Studies:")
for s in optuna.study.get_all_study_summaries(storage):
    print(s.study_name, s.datetime_start, s.n_trials, "trials")

In [ ]:
study = optuna.multi_objective.load_study("multi", storage)

In [ ]:
unc = optuna.load_study("unconstrained", storage)
uf = to_df(unc)
print(uf.loc[uf.value.idxmin()])
optuna.visualization.plot_parallel_coordinate(unc)

In [ ]:
tf, fig = get_pareto_front(study, hover=["alpha", "ciCorrection"], color="dz")
fig.show()

In [ ]:
number = 221

start="2020-03-01"
end="2020-06-01"
f = "output-calibration/%d/infections.txt" % number
cmp = "output-compare/infections.txt"

df, hospital, rki = read_data(f, "Berlin", "berlin-hospital.csv", "berlin-cases.csv")

cmp_res = calc_multi_error(cmp, "Berlin", start, end)
cmp, _, _ = read_data(cmp, "Berlin", "berlin-hospital.csv", "berlin-cases.csv")

df = df[(df.date >= start) & (df.date <= end)]
hospital = hospital[(hospital.Datum >= start) & (hospital.Datum <= end)]
rki = rki[(rki.date >= start) & (rki.date <= end)]

print(cmp_res)
tf[tf.number == number]

In [ ]:
ax = df.plot.scatter(x="date", y=["cases"], label=["Auto Calibrated"], logy=True, figsize=(16, 8))
cmp.plot.scatter(x="date", y=["cases"], color="green", label=["Manual"], ax=ax)
rki.plot.scatter(x="date", y=["cases"], color="orange", label=["RKI Cases"], ax=ax)
plt.title("Cases")

In [ ]:
ax = df.plot(x="date", y=["casesSmoothed"], label=["Auto calibrated"], logy=True, figsize=(16, 8))
rki.plot(x="date", y=["casesSmoothed"], label=["RKI cases"], ax=ax)
cmp.plot(x="date", y=["casesSmoothed"], label=["Manual"], ax=ax)
plt.title("Smoothed cases")

In [ ]:
ax = df.plot(x="date", y=["nSeriouslySick", "nCritical"], logy=True, figsize=(16, 8))
hospital.plot(x="Datum", y=["Stationäre Behandlung", "Intensivmedizin"], logy=True, ax=ax)

plt.title("Hospitalization")

In [ ]:
get_parallel_coordinate_plot(study)

In [ ]:
get_contour_plot(study, "alpha", "ciCorrection")